
# K-En Yakın Komşu ( K-Nearest Neighbors )


knn bir classification algoritmasıdır çok basit ama etkili bir yöntemdir mesela koordinat düzleminde evet ve hayır etiketi verdiğimiz noktalar bulunsun biz o düzlemde birtane alanda bir nokta olsaydı bu evetmi hayırmı olurdu şeklinde gösteriyoruz makina o noktanın en yakın komşularına bakıyor mesela 3 tane hayır 2 tane evet noktası gördü ise demekki ben hayır etiketinde olmalıyım diyor

In [1]:
# 1. Exploratory Data Analysis
# 2. Data Preprocessing & Feature Engineering
# 3. Modeling & Prediction
# 4. Model Evaluation
# 5. Hyperparameter Optimization
# 6. Final Model

import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)


################################################
# 1. Exploratory Data Analysis
################################################

df = pd.read_csv("datasets/diabetes.csv") # yine diyabetten devam



################################################
# 2. Data Preprocessing & Feature Engineering
################################################

y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)

X_scaled = StandardScaler().fit_transform(X) # bu numpy array olarak dönüyor o yüzden tekrar etiket isimlerini atamamız lazım

X = pd.DataFrame(X_scaled, columns=X.columns) # x_scaled eskisi gibi etiketli gözükecek

X.head()



,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496


In [2]:
################################################
# 3. Modeling & Prediction
################################################

knn_model = KNeighborsClassifier().fit(X, y)

################################################
# random_user = X.sample(1, random_state=45)     istersen bu şekildede tek bir kişiyi kontrol edebilirsin
# knn_model.predict(random_user)                 
################################################


# Confusion matrix için y_pred:
y_pred = knn_model.predict(X)

# AUC için y_prob:
y_prob = knn_model.predict_proba(X)[:, 1]

print(classification_report(y, y_pred))

# AUC
roc_auc_score(y, y_prob)
# 0.90

              precision    recall  f1-score   support

           0       0.85      0.90      0.87       500
           1       0.79      0.70      0.74       268

    accuracy                           0.83       768
   macro avg       0.82      0.80      0.81       768
weighted avg       0.83      0.83      0.83       768



np.float64(0.9017686567164179)

In [4]:
# yine sonuçlar gayet güzel gözüküyor fakat tekrardan cross validate ile test ve eğitim verisini bi kontrol edelim
cv_results = cross_validate(knn_model, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])

cv_results['test_accuracy'].mean() # 0.73

cv_results['test_f1'].mean() # 0.59

cv_results['test_roc_auc'].mean() # 0.78

# !!!! bu değerler üsttekinden çok farklı yanlılık problemi olabilir overfitting underfitting olabilir
# ama kesin olan şey bizim güveneceğimiz buradaki metrik ortalamalarıdır
# peki ne yapmak lazım bu değerleri arttırmak için
# 1. Örnek boyutu arttıralabilir.
# 2. Veri ön işleme
# 3. Özellik mühendisliği
# 4. İlgili algoritma için optimizasyonlar yapılabilir.

knn_model.get_params() # örnek n_neighbors 5 girili bunu deneye deneye bakacağız 2,3,6,7 hangisinde skor yükseliyorsa


{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [6]:
################################################
# 5. Hyperparameter Optimization
################################################

# şimdi olması gereken en optimum komşuluk sayısını bulalım

knn_model = KNeighborsClassifier() # yine modelimiz

knn_params = {"n_neighbors": range(2, 50)} # 2 den 50 ye kadar neighbors ları arttır


# gridsearchcv en iyi parametre kombinasyonunu bulmak için her kombinasyonu dener ve CV ile değerlendirir
knn_gs_best = GridSearchCV(knn_model, knn_params, cv=5, n_jobs=-1, verbose=1).fit(X, y)
# parametre olarak önce modeli girdik
# sonra modelde kullanılacak parametreyi girdikki 2 den 50 ye kadar hepsini tek tek deneyecek
# sonra cross validation kullanarak 5 katlı doğrulama yaptık
# n_jobs=-1 -> tüm işlemci çekirdeklerini kullanarak işlemi hızlandırdık
# verbose=1 -> işlem sürecini takip edebilmek için detaylı çıktı alıyoruz
# fit(X, y) -> modeli veriye uygula (en iyi n_neighbors'ı bulmak için)

# En iyi komşuluk sayısını görelim
print("En iyi n_neighbors:", knn_gs_best.best_params_)

# En iyi skoru görelim
print("En iyi skor:", knn_gs_best.best_score_)


Fitting 5 folds for each of 48 candidates, totalling 240 fits
En iyi n_neighbors: {'n_neighbors': 17}
En iyi skor: 0.7669892199303965


In [10]:
# şimdide son hali
knn_final = knn_model.set_params(**knn_gs_best.best_params_).fit(X, y) # ** sayesinde best_params_ daki parametreleri buraya uyguluyoruz

cv_results = cross_validate(knn_final,
                            X,
                            y,
                            cv=5,
                            scoring=["accuracy", "f1", "roc_auc"])

cv_results['test_accuracy'].mean() # 0.766
cv_results['test_f1'].mean() # 0.617
cv_results['test_roc_auc'].mean() # 0.812
# yani önceki çıktıları kontrol edince görülüyorki modeli optimize ettik

np.float64(0.8127938504542278)